In [22]:
import pyltr
import numpy as np
import sklearn.tree
import random
import math
import os
import pickle

In [29]:
NUM_TREES = 50
DATA_FOLDER = 'data/'
EXPERIMENTS = ['long-tail','noisy','query-length/c1','query-length/c2','query-length/c3','']
#EXPERIMENTS  = ['']

NUM_EXPERIMENTS = 10
MODES = ['train','val','test']
feature_idxes = [19,4]



In [30]:
file_dict = {
    'long-tail' :{
        'train': 'train_longtail_suggest_matrix.npz',
        'val': 'val_longtail_suggest_matrix.npz',
        'test': 'test_longtail_suggest_matrix.npz',
    },
    'noisy' :{
        'train': 'tr_noisy_suggest_matrix.npz',
        'val': 'val_noisy_suggest_matrix.npz',
        'test': 'test_noisy_suggest_matrix.npz',
    },
    'query-length/c1' : {
        'train': 'tr_c1_suggest_matrix.npz',
        'val': 'val_c1_suggest_matrix.npz',
        'test': 'test_c1_suggest_matrix.npz',
    },
    'query-length/c2' : {
        'train': 'tr_c2_suggest_matrix.npz',
        'val': 'val_c2_suggest_matrix.npz',
        'test': 'test_c2_suggest_matrix.npz',
    },
    'query-length/c3': {
        'train': 'tr_c3_suggest_matrix.npz',
        'val': 'val_c3_suggest_matrix.npz',
        'test': 'test_c3_suggest_matrix.npz',
    },
    '' : {
        'train': 'train_suggest_matrix.npz',
        'val': 'val_suggest_matrix.npz',
        'test': 'test_suggest_matrix.npz',
    }
}

#col0=anchor ID, col1=suggestion ID. col 2-19 features, col 19 is scores
def make_sessions_ids(anchor_ids):
    
    sessions_ids = np.zeros([len(anchor_ids)])
    iters = len(anchor_ids) /20
    for i in range(iters):
        sessions_ids[i*20:(i+1)*20] = i
    return sessions_ids

def make_data(data_folder, experiment, file_dict,mode ,feature_idx=20):
            
        path = os.path.join(data_folder, experiment, file_dict[experiment][mode] )
        
        print path
        data_train = np.load(path)['arr_0']
       
        data_anchor_ids = data_train[:,0]
        data_suggestion_ids = data_train[:,1]
        data_features = data_train[:,3:feature_idx]
        data_label_scores = data_train[:,20]
        data_session_ids = make_sessions_ids(data_anchor_ids)
        
        return data_anchor_ids, data_suggestion_ids, data_features, data_label_scores, data_session_ids
    
def MRR_Score(Epred, test_data_session_ids, test_data_label_scores, session_length=20):
    sessions = len(test_data_session_ids)/session_length
    idx_array = []

    for session_idx in range(0, sessions):
        predicted_scores = Epred[session_idx*session_length:(session_idx +1)*session_length]
        labels = test_data_label_scores[session_idx*session_length:(session_idx +1)*session_length]
        merged = np.array([predicted_scores, labels]).T
        
        merged = merged[merged[:, 0].argsort()]

        pred_idx = np.argmax(merged[:,1][::-1]) 
        idx_array.append(pred_idx)


    sum_score = 0
    for i,val in enumerate(idx_array): 
        sum_score += (1./(val+1))
    return sum_score / sessions




In [4]:
"""
Some small demo's
metric = pyltr.metrics.NDCG(k=20)

model = pyltr.models.LambdaMART(
    metric=metric,
    n_estimators=n_trees,
    learning_rate=1./n_trees,
    max_features=0.3,
    query_subsample=(1./n_trees),
    max_leaf_nodes=10,
    min_samples_leaf=200,
    verbose=1,
    random_state=None
)           

monitor = pyltr.models.monitors.ValidationMonitor(
    val_data_features, val_data_label_scores, val_data_session_ids, metric=metric, stop_after=5)
    
    model.fit(train_data_features, train_data_label_scores, train_data_session_ids)
    
    Epred = model.predict(test_data_features)

print 'Random ranking:', metric.calc_mean_random(val_data_session_ids, val_data_label_scores)
print 'Our model:', metric.calc_mean(test_data_session_ids, test_data_label_scores, Epred)

print MMR_Score(Epred, test_data_session_ids, test_data_label_scores)

"""



"\nSome small demo's\nmetric = pyltr.metrics.NDCG(k=20)\n\nmodel = pyltr.models.LambdaMART(\n    metric=metric,\n    n_estimators=n_trees,\n    learning_rate=1./n_trees,\n    max_features=0.3,\n    query_subsample=(1./n_trees),\n    max_leaf_nodes=10,\n    min_samples_leaf=200,\n    verbose=1,\n    random_state=None\n)           \n\nmonitor = pyltr.models.monitors.ValidationMonitor(\n    val_data_features, val_data_label_scores, val_data_session_ids, metric=metric, stop_after=5)\n    \n    model.fit(train_data_features, train_data_label_scores, train_data_session_ids)\n    \n    Epred = model.predict(test_data_features)\n\nprint 'Random ranking:', metric.calc_mean_random(val_data_session_ids, val_data_label_scores)\nprint 'Our model:', metric.calc_mean(test_data_session_ids, test_data_label_scores, Epred)\n\nprint MMR_Score(Epred, test_data_session_ids, test_data_label_scores)\n\n"

In [32]:
metric = pyltr.metrics.NDCG(k=20)

for feature_idx in feature_idxes:
    result_dict = {
        'long-tail' :[],
        'noisy' :[],
        'query-length/c1' : [],
        'query-length/c2' : [],
        'query-length/c3': [],
        '' : []
    }
    for experiment in EXPERIMENTS:
        for epoch in range(NUM_EXPERIMENTS):
            print "--- new experiment ---" 
            train_data_anchor_ids, train_data_suggestion_ids, train_data_features, \
            train_data_label_scores, train_data_session_ids =  make_data(DATA_FOLDER, experiment, file_dict,'train' ,feature_idx=feature_idx)
            val_data_anchor_ids, val_data_suggestion_ids, val_data_features, \
            val_data_label_scores, val_data_session_ids =  make_data(DATA_FOLDER, experiment, file_dict,'val' ,feature_idx=feature_idx) 

            test_data_anchor_ids, test_data_suggestion_ids, test_data_features, \
            test_data_label_scores, test_data_session_ids =  make_data(DATA_FOLDER, experiment, file_dict,'test' ,feature_idx=feature_idx) 

            model = pyltr.models.LambdaMART(
                metric=metric,
                n_estimators=NUM_TREES,
                learning_rate=1./NUM_TREES,
                max_features=0.3,
                query_subsample=(1./NUM_TREES),
                max_leaf_nodes=10,
                min_samples_leaf=200,
                verbose=0,
                random_state=None
            )

            model.fit(train_data_features, train_data_label_scores, train_data_session_ids)

            Epred = model.predict(test_data_features)

            random_ranking = metric.calc_mean_random(val_data_session_ids, val_data_label_scores)
            model_ndcg = metric.calc_mean(test_data_session_ids, test_data_label_scores, Epred)
            mrr_score = MRR_Score(Epred, test_data_session_ids, test_data_label_scores)
            print "Experiment: ", epoch + 1
            print "MMR Score for " + experiment + " : ", mrr_score
            result_dict[experiment].append(mrr_score)
    pickle.dump( result_dict, open( 'results/' + str(feature_idx)+ '_' + str(experiment) + "results_save.p", "wb" ) )
    print "__________________________________________________"

--- new experiment ---
data/long-tail/train_longtail_suggest_matrix.npz
data/long-tail/val_longtail_suggest_matrix.npz
data/long-tail/test_longtail_suggest_matrix.npz
Experiment:  1
MMR Score for long-tail :  0.43311105393
--- new experiment ---
data/long-tail/train_longtail_suggest_matrix.npz
data/long-tail/val_longtail_suggest_matrix.npz
data/long-tail/test_longtail_suggest_matrix.npz
Experiment:  2
MMR Score for long-tail :  0.41355242083
--- new experiment ---
data/long-tail/train_longtail_suggest_matrix.npz
data/long-tail/val_longtail_suggest_matrix.npz
data/long-tail/test_longtail_suggest_matrix.npz
Experiment:  3
MMR Score for long-tail :  0.405854653196
--- new experiment ---
data/long-tail/train_longtail_suggest_matrix.npz
data/long-tail/val_longtail_suggest_matrix.npz
data/long-tail/test_longtail_suggest_matrix.npz
Experiment:  4
MMR Score for long-tail :  0.465006915444
--- new experiment ---
data/long-tail/train_longtail_suggest_matrix.npz
data/long-tail/val_longtail_sugges

KeyboardInterrupt: 

In [10]:
"""
ADJ model implementation
"""





val_data_anchor_ids, val_data_suggestion_ids, val_data_features, \
val_data_label_scores, val_data_session_ids =  make_data(DATA_FOLDER, '', file_dict,'train' ,feature_idx=4) 

test_data_anchor_ids, test_data_suggestion_ids, test_data_features, \
test_data_label_scores, test_data_session_ids =  make_data(DATA_FOLDER, 'query-length/c3', file_dict,'test' ,feature_idx=4)

print val_data_label_scores.shape
print test_data_label_scores.shape


data/train_suggest_matrix.npz
data/query-length/c3/test_c3_suggest_matrix.npz
(359200,)
(146260,)


In [12]:

"""
def MMR_Score(Epred, test_data_session_ids, test_data_label_scores, session_length=20):
    sessions = len(test_data_session_ids)/session_length
    idx_array = []

    for session_idx in range(0, sessions):
        predicted_scores = Epred[session_idx*session_length:(session_idx +1)*session_length]
        labels = test_data_label_scores[session_idx*session_length:(session_idx +1)*session_length]
        merged = np.array([predicted_scores, labels]).T
        
        merged = merged[merged[:, 0].argsort()]

        pred_idx = np.argmax(merged[:,1][::-1]) 
        idx_array.append(pred_idx)
 
    idx_array = np.histogram(idx_array)[0]

    sum_score = 0
    for i,val in enumerate(idx_array):
        print (1./(i+1)) * val
        sum_score += (1./(i+1)) * val
    return sum_score / sessions

"""

print MRR_Score(val_data_features[:,0], val_data_session_ids,  val_data_label_scores)
print MRR_Score(test_data_features[:,0], test_data_session_ids,  test_data_label_scores)

0.30775179996
0.310757548371


In [ ]:
labels  = [1,0,0,0,1,0,0,0,1]
test_data_session_ids = [1,1,1,2,2,2,3,3,3]
features = [9,1,1,9,3,1,3,2,1]

In [ ]:
MMR_Score(features, test_data_session_ids,  labels, session_length=3)

In [21]:
result_dict

{'': [0.5405383144324033,
  0.5399769710790338,
  0.550276722372385,
  0.5442619284362216,
  0.5488963220814814,
  0.537359104769452,
  0.5498841321213621,
  0.5457360616759168,
  0.5485279752534448,
  0.5489958412994321],
 'long-tail': [0.5086193641149739,
  0.5192673730153502,
  0.5087087792420795,
  0.5139729755973782,
  0.5209165904552708,
  0.5303805969797115,
  0.49766256200264025,
  0.48548909549391805,
  0.4870286420621442,
  0.4917389813151992],
 'noisy': [0.5443452907631605,
  0.5486075093736763,
  0.5445594912715378,
  0.5435193089131723,
  0.5391056944870003,
  0.5459361305351251,
  0.5372110274901023,
  0.5420938292981136,
  0.534830523908856,
  0.542607522451403],
 'query-length/c1': [0.541545965265918,
  0.5392176245051008,
  0.5392592597939515,
  0.5323738830713733,
  0.5337458503060749,
  0.5422357685473888,
  0.5459774901251432,
  0.5375598411738897,
  0.5352421772328387,
  0.5410496106741213],
 'query-length/c2': [0.5469503800815595,
  0.5400502085841963,
  0.5401652